# Logistic regression with `tf.placeholder`

In [ ]:
""" Solution for simple logistic regression model for MNIST
with placeholder
MNIST dataset: yann.lecun.com/exdb/mnist/
Created by Chip Huyen (huyenn@cs.stanford.edu)
CS20: "TensorFlow for Deep Learning Research"
cs20.stanford.edu
Lecture 03
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

## Hyperparameter setting

In [ ]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

## Data reading

In [ ]:
# Step 1: Read in data
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data / 255.
train_labels = np.asarray(train_labels, dtype=np.int32)

test_data = test_data / 255.
test_labels = np.asarray(test_labels, dtype=np.int32)

In [ ]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
def next_batch(data, labels, batch_size):
  # TODO
  return X_batch, Y_batch

## Phase 1: Build a graph

In [ ]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# each lable is one hot vector.
X = tf.placeholder(tf.float32, [batch_size, 784], name='image') 
Y = tf.placeholder(tf.int32, [batch_size, 10], name='label')

In [ ]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer())
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

In [ ]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(X, w) + b 

In [ ]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(entropy) # computes the mean over all the examples in the batch
# loss = tf.reduce_mean(-tf.reduce_sum(tf.nn.softmax(logits) * tf.log(Y), reduction_indices=[1]))

In [ ]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [ ]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

## Phase 2: Train a model using `tf.Session`

In [ ]:
writer = tf.summary.FileWriter('./graphs/02-1.logreg_placeholder', tf.get_default_graph())
with tf.Session() as sess:
  start_time = time.time()
  sess.run(tf.global_variables_initializer())
  n_batches = int(mnist.train.num_examples/batch_size)

  # train the model n_epochs times
  for i in range(n_epochs): 
    total_loss = 0

    for j in range(n_batches):
      X_batch, Y_batch = mnist.train.next_batch(batch_size)
      _, loss_batch = sess.run([optimizer, loss], {X: X_batch, Y:Y_batch}) 
      total_loss += loss_batch
    print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
  print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0

  for i in range(n_batches):
    X_batch, Y_batch = mnist.test.next_batch(batch_size)
    accuracy_batch = sess.run(accuracy, {X: X_batch, Y:Y_batch})
    total_correct_preds += accuracy_batch

  print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

writer.close()